In [ ]:
%load_ext autoreload
%autoreload 2

from modules.auth import *
from modules.assessments_endpoints import *
from modules.frame_transformations import *
from modules.config import base_url_illuminate
from airflow import DAG
from airflow.operators.python import PythonOperator
import logging
import os

os.makedirs('logs', exist_ok=True)
logging.basicConfig(filename='./logs/Illuminate_operations.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S', force=True)
logging.info('\n\n-------------New Illuminate Operations Logging Instance')


access_token, expires_in = get_access_token()

#For testing
assessment_id_list = [115519, 141498]
temp = get_all_assessments_metadata(access_token)
assessment_id_list = list(temp['assessment_id'].unique())
missing_ids_from_metadata = ['114845', '141498'] # Add assessments that are not present in assessements metadata
assessment_id_list = list(set(assessment_id_list + missing_ids_from_metadata))


test_results_group, log_results_group = loop_through_assessment_scores(access_token, assessment_id_list, 'Group') #Seperate table
test_results_standard, log_results_standard = loop_through_assessment_scores(access_token, assessment_id_list, 'Standard')
test_results_no_standard, log_results_no_standard = loop_through_assessment_scores(access_token, assessment_id_list, 'No_Standard')

test_results_combined = bring_together_test_results(test_results_no_standard, test_results_standard)
test_results_view = create_test_results_view(test_results_combined, '23-24') #add in grade level col, string matching


# access_token, expires_in = get_access_token()








In [ ]:
import pandas as pd
c =pd.read_csv('/home/g2015samtaylor/illuminate/assessment_results_combined.csv')
v =pd.read_csv('/home/g2015samtaylor/illuminate/assessment_results_view.csv')
sr =pd.read_table('/home/icef/powerschool/Student_Rosters.txt')

In [ ]:
sub = v.loc[v['grade'].isin([0,1,2,3,4,5])][['title', 'curriculum']].drop_duplicates()


# for 6th grade science i'm noticing that the scores in big query are different than what i see on illuminate

# So is it safe to say anytime 'ELA' appears in the title column I can put 'Into Reading' for curriculum, 
# and anytime 'Math' appears in the title I can put 'IM' for curriculum?

In [ ]:
sub['title'].unique()

In [ ]:
v.loc[(v['local_student_id'] == 44319) & (v['assessment_id'] == 115939)]

In [ ]:
o, l = get_assessment_scores(access_token, '115939', 'Standard')
o.loc[o['local_student_id'] == '43977'][['assessment_id', 'title','standard_code', 'local_student_id', 'points', 'points_possible', 'percent_correct']]